In [1]:
dir = pwd()
cd("$dir\\input")

type parr
    n::Int
    m::Int
    rjesenje::Int
end

In [2]:
function openNth(br,sadrzaj)
    filein = open(sadrzaj[br])
    puni=""
    for line in eachline(filein)
        puni = puni * line
    end
    puni = strip(puni)
    #println(puni)
    close(filein)
    return puni
end


function parsiraj!(puni,c,A,b,par::parr)
    brojevi_str = split(puni,r" |\r|\n",keep=false)
    par.n = parse(Int,brojevi_str[1])
    par.m = parse(Int,brojevi_str[2])
    par.rjesenje = parse(Int,brojevi_str[3])
    
    cA = brojevi_str[4:(3+par.n)]
    tempA = brojevi_str[4+par.n:(3+par.n+par.n*par.m)]
    bA = brojevi_str[4+par.n+par.n*par.m:(3+par.n+par.n*par.m+par.m)]
    
    for i in bA
        ii = parse(Float64,i)
        push!(b,ii)
    end
    for i in cA
        ii = parse(Float64,i)
        push!(c,ii)
    end
    
    for poc in 1:par.n:(par.m*par.n)
        #println(poc)
        tt = []
        for i in 0:(par.n-1)
            #print(tempA[poc+i]*" ")
            push!(tt,parse(Float64,tempA[poc+i]))
        end

        push!(A,map(Float64,tt))
    end
end

parsiraj! (generic function with 1 method)

In [3]:
type Individua
    Duzina
    Hromozom
    Fitnes::Float64
end
function Individua(Duzina::Int)
    Hromozom = BitArray(Duzina)
    rand!(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end
function Individua(Hromozom::BitArray)
    Duzina = length(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end
function Mutiraj!(a::Individua)
    poz = rand(1:a.Duzina)
    a.Hromozom[poz] = !a.Hromozom[poz]
    return a
end
function Ukrstanje(a::Individua,b::Individua,p::Float64)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2];a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2];b.Hromozom[(tacka2+1):end]])
    if(p>rand())
        return t1
    end
    return t2
end
function Ukrstanje(a::Individua,b::Individua)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2];a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2];b.Hromozom[(tacka2+1):end]])

    return [t1;t2]
end

function UkrstanjeN(a::Individua,b::Individua,p::Float64,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    #println(niz)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    #println(h1)
    for i in 1:2:(kraj-1)
        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        #println("tacke[$i] = $(tacke[i]), tacke[$i+1] = $(tacke[i+1]), tacke[$i+2] = $(tacke[i+2])")
        #a.Hromozom[tacka[i]:tacka[i+1]];b.Hromozom[(tacka[i+1]+1):tacka[i+2]]
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    if(p>rand())
        return t1
    end
    return t2
end
function UkrstanjeN(a::Individua,b::Individua,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    #println(niz)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    #println(h1)
    for i in 1:2:(kraj-1)
        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        #println("tacke[$i] = $(tacke[i]), tacke[$i+1] = $(tacke[i+1]), tacke[$i+2] = $(tacke[i+2])")
        #a.Hromozom[tacka[i]:tacka[i+1]];b.Hromozom[(tacka[i+1]+1):tacka[i+2]]
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    return [t1;t2]
end

function UInt(niz)
    rez=Array(Float64,0)
    for a in niz
        push!(rez,Float64(a))
    end
    return rez;
end

UInt (generic function with 1 method)

In [4]:
function Evauliraj!(a::Individua,A,c,b,n,m)
    #a.Fitnes = rand()
    tt=UInt(a.Hromozom)
    fitnes = tt'*c
    kazna = fitnes
    for i in 1:m
        sum = 0.0
        for j in 1:n
            sum += (A[i][j]*tt[j])
        end
        if sum<=b[i]
            #continue
        else
            fitnes-=kazna
        end
    end
    #println(fitnes)
    a.Fitnes = fitnes[1]
end

Evauliraj! (generic function with 1 method)

In [5]:
type Populacija
    DuzinaHromozoma::Int
    MaxGeneracija::Int
    VelicinaPopulacije::Int
    BrElite::Int
    BrMutacija::Int
    Iteracija::Int
    Populacija
end
function Populacija(Duzina::Int64)
    MaxGeneracija = 100*Duzina # 100 * brojvarijabli
    VelicinaPopulacije = 51
    BrElite = 1;
    BrMutacija = 15;
    Iteracija = 0;
    populacija = [];
    for a in 1:VelicinaPopulacije
        push!(populacija,Individua(Duzina))
    end
    return Populacija(Duzina,MaxGeneracija,VelicinaPopulacije,BrElite,BrMutacija,Iteracija,populacija)
end
function EvalPop!(pop::Populacija,m,n,A,b,c)
    for a in pop.Populacija
        Evauliraj!(a,A,c,b,n,m)
    end
    sort!(pop.Populacija, by=x->(x.Fitnes), rev=true)
end
function Turnir(pop::Populacija,m::Int64,p::Float64)
    rez = [];
    for a in 1:m
        temp = [];
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        sort!(temp,by=x->(x.Fitnes))
        if(p<rand())
            push!(rez,temp[1])
        else
            push!(rez,temp[2])
        end
    end
    return rez
end

Turnir (generic function with 1 method)

In [6]:
function DajElite(pop::Populacija)
    br = pop.BrElite
    return pop.Populacija[1:br]
end
function Mutacija(pop::Populacija,djeca)
    for i in 1:pop.BrMutacija
        br = rand(1:length(djeca))
        #println("$i = $br")
        d = djeca[br]
        t = Mutiraj!(d)
        djeca[br] = t;
        #print(".")
    end
end
function ProsjekFitnesa(pop::Populacija)
    suma = 0;
    for a in pop.Populacija 
        suma+=a.Fitnes 
    end
    suma/=length(pop.Populacija)
    return suma
end

ProsjekFitnesa (generic function with 1 method)

In [7]:
function DajDjecu(pop::Populacija,roditelji)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        r1 = roditelji[rand(1:length(roditelji))]
        r2 = roditelji[rand(1:length(roditelji))]
        #d = UkrstanjeN(r1,r2,11)
        d = Ukrstanje(r1,r2)
        #println("d: $d \n\n")
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end

DajDjecu (generic function with 1 method)

In [8]:
function UInt(br::Int,m::Int)
    rez=Array(Float64,0)
    for a in 1:m
        t = br&1
        #println("$br>>>1 = $t")
        push!(rez,t)
        br = br>>>1
    end
    return rez;
end
function Evauliraj!(br,A,c,b,n,m)
    print('eval');
    #a.Fitnes = rand()
    tt=UInt(br,n)
    #print(".")
    #println("$tt \n$c")
    fitnes = tt'*c
    kazna = fitnes
    
    for i in 1:m
        sum = 0.0
        for j in 1:n
            sum += (A[i][j]*tt[j])
        end
        if sum<=b[i]
            #continue
        else
            fitnes-=kazna
        end
    end
    
    return fitnes[1]
end

LoadError: LoadError: syntax: invalid character literal
while loading In[8], in expression starting on line 12

In [9]:
sadrzaj = readdir()

max = 0;
maxhistorija = []
    c = Array(Float64,0); b= Array(Float64,0)
    A = []
n = 0
m = 0
rjesenje = 0
for file in 1:100
    println("-- $file --")
    puni = openNth(1,sadrzaj)
    c = Array(Float64,0); b= Array(Float64,0)
    A = []
    par = parr(0,0,0)
    parsiraj!(puni,c,A,b,par)
    n = par.n; m= par.m; rjesenje = par.rjesenje
    
 #           b = [5]
  #      A = [2,3,4,5]
   #     c = [3,4,5,6]
    
  #  n = 4
   # m = 1
    
    
    println("n = $(n) broj objekata, dimenzije hromozoma\nm = $(m) broj dimenzija\nrjesenje = $rjesenje\n")
    
    #println("\nc = $c\nb = $b\nA = $A\n dim= ")
    
    #for k in 1:32767
   #     r = Evauliraj!(k,A,c,b,n,m)
    #    if(r==4015)
    #        println("za $k je $r")
    #    end
  #  end
    
    
    
    
    #inicijalna populacija
    pop = Populacija(n)
    
    for pop.Iteracija in 1:pop.MaxGeneracija
       #break
        #print(".")


        
        EvalPop!(pop::Populacija,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(max<naj)
            max = naj
        end
        

        
        if(pop.Iteracija==pop.MaxGeneracija)
            println("Prosjek fitnesa $prosjek, najbolji fitnes $naj\n\n")
        end
                      
        if(naj==rjesenje)
            println("Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n\n")
            break;
        end

        #println("- populacija $pop\n")

        elite = DajElite(pop)

        #println("- elite $elite\n")

        roditelji = Turnir(pop,Int((pop.VelicinaPopulacije-pop.BrElite)/2),0.9)

        #println("- roditelji $roditelji\n")

        djeca = DajDjecu(pop,roditelji)

        #println("- djeca $djeca\n")

        Mutacija(pop,djeca)

        #println("- djeca $djeca\n")

        #novagen = 
        append!(djeca,elite)

        #println("- nova gen $djeca\n")
        
        pop.Populacija = djeca

    
        #println(pop.Iteracija)
        #break
    end
    
    
    
    break
end

println("Max $max")



-- 1 --
n = 15 broj objekata, dimenzije hromozoma
m = 10 broj dimenzija
rjesenje = 4015

Max 3665.0
